In [1]:
# importar librerias
import numpy as np
from scipy import optimize

In [2]:
# leyendo el data set
data = np.genfromtxt('winequality-red.csv',delimiter=',')

# separando los datos para X y Y
trainPorcen = int(len(data)/100*90)
testPorcen = int(len(data)/100*10)
# print(trainPorcen, testPorcen)
X_train = data[:trainPorcen, :11]
y_train = data[:trainPorcen, 11]

y_train = np.array([int(e) for e in y_train])
y_train = np.squeeze(y_train)

m = y_train.size

print(X_train.shape)
print(y_train.shape)
# print(X_test.shape)
# print(y_test.shape)

(1439, 11)
(1439,)


In [3]:
# definiendo el numero de caracteristicas y numero de etiquetas
input_layer_size = 11
num_labels = 6

In [4]:
#  Calcula la sigmoide de z.
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

In [5]:
# fucion de costo
def lrCostFunction(theta, X, y, lambda_):
    # Inicializa algunos valores utiles
    m = y.size
    
    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)
    
    J = 0
    grad = np.zeros(theta.shape)
    
    h = sigmoid(X.dot(theta.T))
    
    temp = theta
    temp[0] = 0
    
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
    
    grad = (1 / m) * (h - y).dot(X) 
    grad = grad + (lambda_ / m) * temp

    return J, grad

In [6]:
# funcion de one vs all
def oneVsAll(X, y, num_labels, lambda_):

    m, n = X.shape
    
    all_theta = np.zeros((num_labels, n + 1))
#     print(all_theta.shape)
    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)


    for c in np.arange(num_labels):
#         print(c+3)
        initial_theta = np.zeros(n+1)
#         print(initial_theta.shape)
        options = {'maxiter': 1000}
        res = optimize.minimize(lrCostFunction, 
                                initial_theta, 
                                (X, (y == (c + 3)), lambda_), 
                                jac=True, 
                                method='Newton-CG',
                                options=options) 
        
        all_theta[c] = res.x

    return all_theta

In [7]:
lambda_ = 0.0003
all_theta = oneVsAll(X_train, y_train, num_labels, lambda_)

In [8]:
print(all_theta)

[[-3.87589377e+01  1.27050673e+00  8.62306146e+00  3.87215304e+00
  -7.64399279e-02  2.19916402e+01  1.99722884e-01 -1.11001634e-01
  -1.24754504e+01  1.50100653e+01 -5.00160581e+00 -2.06680263e+00]
 [ 1.91486282e+00 -4.29189468e-02  3.50251700e+00  1.81009403e-01
   2.03475331e-01  2.49165486e+00 -1.76639664e-02 -8.25262729e-03
  -1.07146058e+01  1.77202243e+00 -9.04752520e-01 -1.82237393e-01]
 [-6.74349818e+01 -1.81509732e-01  2.07655437e+00  8.95227747e-01
  -7.91528420e-02  2.18022152e+00 -2.05704758e-02  1.90019460e-02
   7.93175426e+01 -6.40360698e-01 -2.39373983e+00 -8.41356627e-01]
 [-5.04378242e-01  7.01026768e-02 -1.93500230e+00 -1.56780832e+00
  -4.39088782e-02 -3.04015461e-01  2.37448140e-02 -1.28198440e-02
  -4.79188186e-01 -2.14599328e-01  7.78874542e-01  1.86615327e-01]
 [ 4.84193450e+01  1.22857181e-01 -3.01458431e+00  5.40930282e-01
   1.72120797e-01 -7.01472554e+00  1.15591507e-02 -1.65043176e-02
  -6.12619297e+01  2.54905007e-01  3.03510271e+00  7.87563874e-01]
 [ 4.

In [9]:
# funcion de prediccion de one vs all
def predictOneVsAll(all_theta, X):
    m = X.shape[0]
    print(m)
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p + 3

In [10]:
print(y_train.shape)
pred = predictOneVsAll(all_theta, X_train)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y_train) * 100))
X_test = data[trainPorcen: , :11]
y_test = data[trainPorcen: , 11]
# print(X_test.shape)
# print(len(X_test))

X_test = np.concatenate((np.ones((len(X_test), 1)), X_test), axis=1)

p = np.argmax(sigmoid(X_test.dot(all_theta.T)), axis = 1)
print('Precision del conjuto de test: {:.2f}%'.format(np.mean(p+3 == y_test) * 100))
print(p + 3)

(1439,)
1439
Precision del conjuto de entrenamiento: 60.39%
Precision del conjuto de test: 57.50%
[5 6 5 6 6 5 5 6 5 6 6 6 6 6 5 6 6 5 5 6 6 6 5 6 6 5 5 5 5 5 5 5 6 6 6 5 7
 5 7 5 6 5 6 5 6 5 5 5 6 6 6 6 6 6 5 6 6 5 6 5 6 5 5 5 6 6 5 6 6 6 6 6 5 5
 5 5 5 6 6 5 5 6 5 6 5 6 6 5 6 5 6 6 5 6 5 6 5 6 6 6 6 6 6 5 6 6 5 6 6 6 6
 5 5 6 5 6 6 5 6 5 5 5 5 5 5 5 6 6 5 5 6 6 6 5 6 5 6 6 6 6 6 6 6 6 5 6 6 6
 6 6 5 6 6 6 6 6 6 6 6 6]
